## 2RPFS Problem (Cmax objective) - Comparison Tables - SP MIP x SP DP

Before running this, notebook, please run notebook 0.4.

In [1]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle

%matplotlib inline

In [2]:
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO

In [3]:
linestyle_tuple = [
     ('dotted',                (0, (1, 1))),
     ('dashed',                (0, (5, 5))),
     ('densely dashed',        (0, (5, 1))),
     ('dashdotdotted',         (0, (3, 5, 1, 5, 1, 5))),
     ('densely dashdotdotted', (0, (3, 1, 1, 1, 1, 1))),

     ('dashdotted',            (0, (3, 5, 1, 5))),
     ('densely dashdotted',    (0, (3, 1, 1, 1))),
     
     ('loosely dashed',        (0, (5, 10))),
     ('loosely dashdotted',    (0, (3, 10, 1, 10))),
     

     ('loosely dashdotdotted', (0, (3, 10, 1, 10, 1, 10))),
     ('densely dotted',        (0, (1, 1))),
     ('loosely dotted',        (0, (1, 10)))]

In [4]:
# https://stackoverflow.com/questions/51483901/is-there-a-way-to-extend-the-solid-color-background-to-the-full-width-of-the-pag
class ExtendedTextBox(BoxStyle._Base):
    """
    An Extended Text Box that expands to the axes limits 
                        if set in the middle of the axes
    """

    def __init__(self, pad=0.3, width=500.):
        """
        width: 
            width of the textbox. 
            Use `ax.get_window_extent().width` 
                   to get the width of the axes.
        pad: 
            amount of padding (in vertical direction only)
        """
        self.width=width
        self.pad = pad
        super(ExtendedTextBox, self).__init__()

    def transmute(self, x0, y0, width, height, mutation_size):
        """
        x0 and y0 are the lower left corner of original text box
        They are set automatically by matplotlib
        """
        # padding
        pad = mutation_size * self.pad

        # we add the padding only to the box height
        height = height + 2.*pad
        # boundary of the padded box
        y0 = y0 - pad
        y1 = y0 + height
        _x0 = x0
        x0 = _x0 +width /2. - self.width/2.
        x1 = _x0 +width /2. + self.width/2.

        cp = [(x0, y0),
              (x1, y0), (x1, y1), (x0, y1),
              (x0, y0)]

        com = [Path.MOVETO,
               Path.LINETO, Path.LINETO, Path.LINETO,
               Path.CLOSEPOLY]

        path = Path(cp, com)

        return path

### List files in the result folder 

In [5]:
resultfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
rpfs_file = os.path.join(resultfolder, '2RPFS_Cmax_all_results.pkl.gz')
rpfs_sp_mip_file = os.path.join(resultfolder, '2RPFS_Cmax_SP-MIP_all_results.pkl.gz')

### Create the output folder 

In [6]:
outputfolder = os.path.join(os.getcwd(), 'results', 'consolidated')
outputfolder_graph = os.path.join(os.getcwd(), 'results', 'consolidated', 'graphs')
outputfolder_table = os.path.join(os.getcwd(), 'results', 'consolidated', 'tables')
if not os.path.exists(outputfolder_graph):
    os.makedirs(outputfolder_graph)
if not os.path.exists(outputfolder_table):
    os.makedirs(outputfolder_table)
#print('Saving files on folder: ' + outputfolder)

### Process consolidated CSV result files

In [7]:
df_rpfs = pd.read_pickle(rpfs_file)  # Robust PFSP Budget solutions with SP=DP only
df_rpfs_sp_mip = pd.read_pickle(rpfs_sp_mip_file)  # Robust PFSP Budget solutions with SP=MIP only
df_rpfs.drop(columns=['executionId'], inplace=True)
df_rpfs_sp_mip.drop(columns=['executionId'], inplace=True)

**Robust dataframe: calculating new fields.**

In [36]:
df_rpfs['optimal'] = df_rpfs['is_optimal'] & df_rpfs['validated'] & (df_rpfs['gap'] <= 1e-8)
df_rpfs['time_limit'] = 7200.0
df_rpfs['time'] = np.minimum(df_rpfs['time_spent'], df_rpfs['time_limit'])
df_rpfs['gap'] = df_rpfs['gap'] * 100.0
df_rpfs['RobCost_worstcase'] = df_rpfs['cmax_dp']
df_rpfs = df_rpfs.rename(columns={"budget_Gamma": "RobCost_Gamma"})
df_rpfs['sp_type'] = 'DP'
df_rpfs.head(2)

instance_name      ub_name      \
model  n  m alpha seq Gamma1 Gamma2                                  
Wagner 10 2 10    01  20     20      RB0101001.txt  mip_separation   
                             40      RB0101001.txt  mip_separation   

                                    RobCost_Gamma  cmax   \
model  n  m alpha seq Gamma1 Gamma2                        
Wagner 10 2 10    01  20     20          20 20     285.1   
                             40          20 40     285.1   

                                         permutation       time_spent  \
model  n  m alpha seq Gamma1 Gamma2                                     
Wagner 10 2 10    01  20     20      10 9 7 5 3 4 2 6 1 8     6.34      
                             40      10 9 2 4 1 7 6 5 3 8     0.11      

                                     time_to_best_sol  iterations  \
model  n  m alpha seq Gamma1 Gamma2                                 
Wagner 10 2 10    01  20     20            6.34             2       
                             40            0.11             4       

                                     num_visited_solutions  num_improvements  \
model  n  m alpha seq Gamma1 Gamma2                                            
Wagner 10 2 10    01  20     20                2                    1          
                             40                4                    3          

                                     is_optimal  validated       gap      \
model  n  m alpha seq Gamma1 Gamma2                                        
Wagner 10 2 10    01  20     20         True       True     0.000000e+00   
                             40         True       True     1.993806e-12   

                                      lb    cost   cmax_dp  cut_count  \
model  n  m alpha seq Gamma1 Gamma2                                     
Wagner 10 2 10    01  20     20      285.1  285.1   285.1    -100.0     
                             40      285.1  285.1   285.1    -100.0     

                                     optimal  time_limit  time  \
model  n  m alpha seq Gamma1 Gamma2                              
Wagner 10 2 10    01  20     20       True      7200.0    6.34   
                             40       True      7200.0    0.11   

                                     RobCost_worstcase instance_size sp_type  
model  n  m alpha seq Gamma1 Gamma2                                           
Wagner 10 2 10    01  20     20            285.1            10x2        DP    
                             40            285.1            10x2        DP

In [37]:
df_rpfs_sp_mip['optimal'] = df_rpfs_sp_mip['is_optimal'] & df_rpfs_sp_mip['validated'] & (df_rpfs_sp_mip['gap'] <= 1e-8)
df_rpfs_sp_mip['time_limit'] = 7200.0
df_rpfs_sp_mip['time'] = np.minimum(df_rpfs_sp_mip['time_spent'], df_rpfs_sp_mip['time_limit'])
df_rpfs_sp_mip['gap'] = df_rpfs_sp_mip['gap'] * 100.0
df_rpfs_sp_mip['RobCost_worstcase'] = df_rpfs_sp_mip['cmax_dp']
df_rpfs_sp_mip['sp_type'] = 'MIP'
df_rpfs_sp_mip.head(2)

ub_name       cmax   \
model  n   m alpha instance_name Gamma1 Gamma2                           
Wagner 150 2 10    RB1501001.txt 20     20      mip_separation  4501.5   
                                        40      mip_separation  4610.5   

                                                                  permutation                      \
model  n   m alpha instance_name Gamma1 Gamma2                                                      
Wagner 150 2 10    RB1501001.txt 20     20      78 91 110 69 32 28 22 12 43 142 23 40 42 128 1...   
                                        40      124 103 26 76 46 141 146 29 43 91 126 37 71 13...   

                                                time_spent  time_to_best_sol  \
model  n   m alpha instance_name Gamma1 Gamma2                                 
Wagner 150 2 10    RB1501001.txt 20     20        8635.79        8635.79       
                                        40        7853.84        7853.84       

                                                mp_total_time  sp_total_time  \
model  n   m alpha instance_name Gamma1 Gamma2                                 
Wagner 150 2 10    RB1501001.txt 20     20         8451.79        146.84       
                                        40         7756.53         94.26       

                                                iterations  \
model  n   m alpha instance_name Gamma1 Gamma2               
Wagner 150 2 10    RB1501001.txt 20     20         59.0      
                                        40         45.0      

                                                num_visited_solutions  \
model  n   m alpha instance_name Gamma1 Gamma2                          
Wagner 150 2 10    RB1501001.txt 20     20              59.0            
                                        40              45.0            

                                                num_improvements  is_optimal  \
model  n   m alpha instance_name Gamma1 Gamma2                                 
Wagner 150 2 10    RB1501001.txt 20     20            58.0           True      
                                        40            44.0           True      

                                                validated     gap      lb    \
model  n   m alpha instance_name Gamma1 Gamma2                                
Wagner 150 2 10    RB1501001.txt 20     20        True     1.999733  4500.6   
                                        40        True     2.169433  4609.5   

                                                cost   cmax_dp cut_count seq  \
model  n   m alpha instance_name Gamma1 Gamma2                                 
Wagner 150 2 10    RB1501001.txt 20     20     -500.0  4501.5      58     01   
                                        40     -500.0  4610.5      44     01   

                                                optimal  time_limit   time   \
model  n   m alpha instance_name Gamma1 Gamma2                                
Wagner 150 2 10    RB1501001.txt 20     20       False     7200.0    7200.0   
                                        40       False     7200.0    7200.0   

                                                RobCost_worstcase sp_type  
model  n   m alpha instance_name Gamma1 Gamma2                             
Wagner 150 2 10    RB1501001.txt 20     20           4501.5          MIP   
                                        40           4610.5          MIP

In [10]:
df_rpfs.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15366 entries, ('Wagner', 10, 2, 10, 'RB0101001.txt', 20, 20) to ('Wilson', 200, 2, 50, 'RB2005010.txt', 100, 100)
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ub_name                15366 non-null  object 
 1   RobCost_Gamma          15366 non-null  object 
 2   cmax                   15366 non-null  float64
 3   permutation            15366 non-null  object 
 4   time_spent             15366 non-null  float64
 5   time_to_best_sol       15366 non-null  float64
 6   iterations             15366 non-null  int64  
 7   num_visited_solutions  15366 non-null  int64  
 8   num_improvements       15366 non-null  int64  
 9   is_optimal             15366 non-null  bool   
 10  validated              15366 non-null  bool   
 11  gap                    15366 non-null  float64
 12  lb                     15366 non-null  float64
 13  cost             

# Tables

Obtain list of C&CG models, instance types

In [11]:
model_list = df_rpfs.reset_index()['model'].unique().tolist()
print(model_list)

['Wagner', 'Wilson']


Add a new column containing the instance size as string

In [12]:
df_temp = df_rpfs.reset_index()
(df_temp['n'].astype(str) + 'x' + df_temp['m'].astype(str)).unique()

array(['10x2', '20x2', '50x2', '100x2', '150x2', '200x2'], dtype=object)

In [38]:
df_temp['instance_size'] = df_temp['n'].astype(str) + 'x' + df_temp['m'].astype(str)
df_rpfs = df_temp.set_index(['model', 'n', 'm', 'alpha', 'seq', 'Gamma1', 'Gamma2'])
df_rpfs.head(2)

instance_name      ub_name      \
model  n  m alpha seq Gamma1 Gamma2                                  
Wagner 10 2 10    01  20     20      RB0101001.txt  mip_separation   
                             40      RB0101001.txt  mip_separation   

                                    RobCost_Gamma  cmax   \
model  n  m alpha seq Gamma1 Gamma2                        
Wagner 10 2 10    01  20     20          20 20     285.1   
                             40          20 40     285.1   

                                         permutation       time_spent  \
model  n  m alpha seq Gamma1 Gamma2                                     
Wagner 10 2 10    01  20     20      10 9 7 5 3 4 2 6 1 8     6.34      
                             40      10 9 2 4 1 7 6 5 3 8     0.11      

                                     time_to_best_sol  iterations  \
model  n  m alpha seq Gamma1 Gamma2                                 
Wagner 10 2 10    01  20     20            6.34             2       
                             40            0.11             4       

                                     num_visited_solutions  num_improvements  \
model  n  m alpha seq Gamma1 Gamma2                                            
Wagner 10 2 10    01  20     20                2                    1          
                             40                4                    3          

                                     is_optimal  validated       gap      \
model  n  m alpha seq Gamma1 Gamma2                                        
Wagner 10 2 10    01  20     20         True       True     0.000000e+00   
                             40         True       True    -1.993806e-14   

                                      lb    cost   cmax_dp  cut_count  \
model  n  m alpha seq Gamma1 Gamma2                                     
Wagner 10 2 10    01  20     20      285.1  285.1   285.1    -100.0     
                             40      285.1  285.1   285.1    -100.0     

                                     optimal  time_limit  time  \
model  n  m alpha seq Gamma1 Gamma2                              
Wagner 10 2 10    01  20     20       True      7200.0    6.34   
                             40       True      7200.0    0.11   

                                     RobCost_worstcase instance_size  
model  n  m alpha seq Gamma1 Gamma2                                   
Wagner 10 2 10    01  20     20            285.1            10x2      
                             40            285.1            10x2

In [39]:
df_temp = df_rpfs_sp_mip.reset_index()
df_temp['instance_size'] = df_temp['n'].astype(str) + 'x' + df_temp['m'].astype(str)
df_rpfs_sp_mip = df_temp.set_index(['model', 'n', 'm', 'alpha', 'seq', 'Gamma1', 'Gamma2'])
df_rpfs_sp_mip.head(2)

instance_name      ub_name       cmax   \
model  n   m alpha seq Gamma1 Gamma2                                          
Wagner 150 2 10    01  20     20      RB1501001.txt  mip_separation  4501.5   
                              40      RB1501001.txt  mip_separation  4610.5   

                                                        permutation                      \
model  n   m alpha seq Gamma1 Gamma2                                                      
Wagner 150 2 10    01  20     20      78 91 110 69 32 28 22 12 43 142 23 40 42 128 1...   
                              40      124 103 26 76 46 141 146 29 43 91 126 37 71 13...   

                                      time_spent  time_to_best_sol  \
model  n   m alpha seq Gamma1 Gamma2                                 
Wagner 150 2 10    01  20     20        8635.79        8635.79       
                              40        7853.84        7853.84       

                                      mp_total_time  sp_total_time  \
model  n   m alpha seq Gamma1 Gamma2                                 
Wagner 150 2 10    01  20     20         8451.79        146.84       
                              40         7756.53         94.26       

                                      iterations  num_visited_solutions  \
model  n   m alpha seq Gamma1 Gamma2                                      
Wagner 150 2 10    01  20     20         59.0             59.0            
                              40         45.0             45.0            

                                      num_improvements  is_optimal  validated  \
model  n   m alpha seq Gamma1 Gamma2                                            
Wagner 150 2 10    01  20     20            58.0           True       True      
                              40            44.0           True       True      

                                         gap      lb    cost   cmax_dp  \
model  n   m alpha seq Gamma1 Gamma2                                     
Wagner 150 2 10    01  20     20      1.999733  4500.6 -500.0  4501.5    
                              40      2.169433  4609.5 -500.0  4610.5    

                                     cut_count  optimal  time_limit   time   \
model  n   m alpha seq Gamma1 Gamma2                                          
Wagner 150 2 10    01  20     20         58      False     7200.0    7200.0   
                              40         44      False     7200.0    7200.0   

                                      RobCost_worstcase sp_type instance_size  
model  n   m alpha seq Gamma1 Gamma2                                           
Wagner 150 2 10    01  20     20           4501.5          MIP       150x2     
                              40           4610.5          MIP       150x2

Treating errors in the `gap` column

In [14]:
df_rpfs['gap'].describe()

count    1.536600e+04
mean     2.817895e-01
std      1.069857e+00
min     -9.563277e-03
25%      0.000000e+00
50%      0.000000e+00
75%      6.472448e-14
max      1.197022e+01
Name: gap, dtype: float64

In [15]:
df_rpfs['gap'] = df_rpfs['gap'].apply(lambda x: np.abs(x))

In [16]:
df_rpfs['gap'].describe()

count    1.536600e+04
mean     2.817929e-01
std      1.069856e+00
min      0.000000e+00
25%      0.000000e+00
50%      1.443093e-14
75%      9.655448e-14
max      1.197022e+01
Name: gap, dtype: float64

In [40]:
df_rpfs_sp_mip['gap'] = df_rpfs_sp_mip['gap'].apply(lambda x: np.abs(x))

In [41]:
df_rpfs_sp_mip['gap'].describe()

count    6.860000e+02
mean     1.189585e+01
std      6.290617e+01
min      0.000000e+00
25%      1.818375e-12
50%      4.137093e-12
75%      1.204018e-11
max      6.353177e+02
Name: gap, dtype: float64

## Table 2. Performance given all instances (SP=DP)

Model-wise Robust PFSP C&CG performance comparison, given all instances.

* % Best Performance is the percentage of instances solved to optimality where the model achieved shorter execution time, when compared to the other models; 

* % Solved contains the percentage of instances solved within the time limit; 

* % Solved < n x m > represents the percentage of solved instances of size n x m; 

* Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality; 

* Median time is the median execution time, in seconds; 

* Median iterations is the median of the number of iterations performed.

In [45]:
def calculate_perc_best_performance(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_model = df_model[df_model['optimal'] == True]    
    df_model = df_model.set_index(['n', 'm', 'alpha', 'seq', 'Gamma1', 'Gamma2'])
    if len(df_model.index) == 0:
        return np.nan
    
    df_others = df.reset_index()
    df_others = df_others[df_others['model'] != model]
    df_others = df_others[df_others['optimal'] == True]
    if len(df_others.index) == 0:
        return np.nan
    df_best_performance = df_others.groupby(by=['n', 'm', 'alpha', 'seq', 'Gamma1', 'Gamma2']).min()['time']
    df_best_performance = df_best_performance.to_frame()
    if len(df_best_performance.index) == 0:
        return np.nan
    
    df_compare = df_best_performance.join(df_model, how='inner', 
                                                     on=['n', 'm', 'alpha', 'seq', 'Gamma1', 'Gamma2'],
                                                     lsuffix='_best')
    df_compare['time_wins'] = (df_compare['time'] < df_compare['time_best']).astype(int)
    return np.round(100.0 * df_compare['time_wins'].sum() / len(df_compare.index), 2)

In [46]:
def calculate_perc_solved(df, model, instance_type = None, instance_size = None):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_ = df_model
    if instance_type is not None:
        df_ = df_[df_['instance_type'] == instance_type]
    if instance_size is not None:
        df_ = df_[df_['instance_size'] == instance_size]
    if len(df_.index) > 0:
        return np.round(100.0 * len(df_[(df_['optimal'] == True)].index) / len(df_.index), 2)
    else:
        return np.nan

In [47]:
# Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality
def calculate_avg_perc_gap(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    df_model = df_model[df_model['optimal'] == False]
    return np.round(df_model['gap'].mean(), 2)

In [48]:
def calculate_median_time(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['time'].median(), 2)

In [49]:
def calculate_avg_time(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['time'].mean(), 2)

In [50]:
def calculate_std_time(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['time'].std(), 2)

In [51]:
def calculate_median_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].median(), 2)

In [52]:
def calculate_avg_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].mean(), 2)

In [53]:
def calculate_std_iterations(df, model):
    df_model = df.reset_index()
    df_model = df_model[df_model['model'] == model]
    return np.round(df_model['iterations'].std(), 2)

In [54]:
model_stats = dict()
for model in model_list:
    model_stats[model] = dict()
    model_stats[model]['% Best Performance'] = calculate_perc_best_performance(df_rpfs, model)
    model_stats[model]['% Solved'] = calculate_perc_solved(df_rpfs, model)  # given all instances
    df_itype = df_rpfs.reset_index()
    instance_size_list = df_itype['instance_size'].unique().tolist()
    for instance_size in instance_size_list:
        model_stats[model]['% Solved '+ instance_size] = calculate_perc_solved(df_rpfs, model, None, instance_size)
    model_stats[model]['Avg. % gap'] = calculate_avg_perc_gap(df_rpfs, model)
    model_stats[model]['Median time'] = calculate_median_time(df_rpfs, model)
    model_stats[model]['Median iterations'] = calculate_median_iterations(df_rpfs, model)

In [55]:
model_stats

{'Wagner': {'% Best Performance': 81.82,
  '% Solved': 84.87,
  '% Solved 10x2': 100.0,
  '% Solved 20x2': 100.0,
  '% Solved 50x2': 97.44,
  '% Solved 100x2': 80.96,
  '% Solved 150x2': 67.52,
  '% Solved 200x2': 63.46,
  'Avg. % gap': 1.11,
  'Median time': 10.87,
  'Median iterations': 8.0},
 'Wilson': {'% Best Performance': 16.13,
  '% Solved': 68.35,
  '% Solved 10x2': 100.0,
  '% Solved 20x2': 100.0,
  '% Solved 50x2': 85.68,
  '% Solved 100x2': 50.56,
  '% Solved 150x2': 45.5,
  '% Solved 200x2': 35.25,
  'Avg. % gap': 1.23,
  'Median time': 147.63,
  'Median iterations': 7.0}}

In [56]:
model_stats_df = pd.DataFrame.from_dict(model_stats)
model_stats_df.T

,% Best Performance,% Solved,% Solved 10x2,% Solved 20x2,% Solved 50x2,% Solved 100x2,% Solved 150x2,% Solved 200x2,Avg. % gap,Median time,Median iterations
Wagner,81.82,84.87,100.0,100.0,97.44,80.96,67.52,63.46,1.11,10.87,8.0
Wilson,16.13,68.35,100.0,100.0,85.68,50.56,45.50,35.25,1.23,147.63,7.0


In [57]:
# Save output table as HTML
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open(os.path.join(outputfolder_table, '2RPFS_model_stats.html'), 'w') as f:
    f.write(html_string.format(table=model_stats_df.T.to_html(classes='mystyle')))

## Table 3. Performance per instance group and model (SP=DP)

Model-wise Robust PFSP C&CG performance comparison, per instance group.

* % Best Performance is the percentage of instances solved to optimality where the model achieved shorter execution time, when compared to the other models; 

* % Solved contains the percentage of instances solved within the time limit; 

* Avg. % Gap is the average percentage gap of solutions from instances not solved to optimality; 

* Avg. time and Std. dev. of time are the mean and standard deviation in solution time (s), respectively;

* Avg. iterations and Std. dev. of iterations are the mean and standard deviation of the number of iterations performed.

In [65]:
per_instance_stats = dict()
df_itype = df_rpfs.reset_index()
instance_size_list = df_itype['instance_size'].unique().tolist()
sp_type = 'DP'
for instance_size in instance_size_list:
    print('Process instance_size = ', instance_size)
    df_instance = df_itype[df_itype['instance_size'] == instance_size]
    for model in model_list:
        per_instance_stats[(sp_type,instance_size,model)] = dict()
        per_instance_stats[(sp_type,instance_size,model)]['% Best Performance'] = calculate_perc_best_performance(df_instance, model)
        per_instance_stats[(sp_type,instance_size,model)]['% Solved'] = calculate_perc_solved(df_rpfs, model, None, instance_size)
        per_instance_stats[(sp_type,instance_size,model)]['Avg. % gap'] = calculate_avg_perc_gap(df_instance, model)
        per_instance_stats[(sp_type,instance_size,model)]['Avg. time'] = calculate_avg_time(df_instance, model)
        per_instance_stats[(sp_type,instance_size,model)]['Std. dev. of time'] = calculate_std_time(df_instance, model)
        per_instance_stats[(sp_type,instance_size,model)]['Avg. iterations'] = calculate_avg_iterations(df_instance, model)
        per_instance_stats[(sp_type,instance_size,model)]['Std. dev. of iterations'] = calculate_std_iterations(df_instance, model)

Process instance_size =  10x2
Process instance_size =  20x2
Process instance_size =  50x2
Process instance_size =  100x2
Process instance_size =  150x2
Process instance_size =  200x2


In [66]:
per_instance_stats

{('DP', '10x2', 'Wagner'): {'% Best Performance': 56.56,
  '% Solved': 100.0,
  'Avg. % gap': nan,
  'Avg. time': 0.28,
  'Std. dev. of time': 0.8,
  'Avg. iterations': 4.04,
  'Std. dev. of iterations': 2.47},
 ('DP', '10x2', 'Wilson'): {'% Best Performance': 36.24,
  '% Solved': 100.0,
  'Avg. % gap': nan,
  'Avg. time': 0.26,
  'Std. dev. of time': 0.58,
  'Avg. iterations': 3.73,
  'Std. dev. of iterations': 2.21},
 ('DP', '20x2', 'Wagner'): {'% Best Performance': 80.48,
  '% Solved': 100.0,
  'Avg. % gap': nan,
  'Avg. time': 2.09,
  'Std. dev. of time': 6.56,
  'Avg. iterations': 7.37,
  'Std. dev. of iterations': 7.77},
 ('DP', '20x2', 'Wilson'): {'% Best Performance': 19.12,
  '% Solved': 100.0,
  'Avg. % gap': nan,
  'Avg. time': 7.06,
  'Std. dev. of time': 53.7,
  'Avg. iterations': 6.98,
  'Std. dev. of iterations': 8.26},
 ('DP', '50x2', 'Wagner'): {'% Best Performance': 94.94,
  '% Solved': 97.44,
  'Avg. % gap': 0.43,
  'Avg. time': 309.79,
  'Std. dev. of time': 1303.1,

In [67]:
# https://stackoverflow.com/questions/57606801/pandas-style-options-to-latex

In [68]:
pd.set_option('display.max_columns', None)
df_table3 = pd.DataFrame.from_dict(per_instance_stats)
df_table3

DP                                              \
                          10x2            20x2            50x2              
                         Wagner  Wilson  Wagner  Wilson  Wagner   Wilson    
% Best Performance        56.56   36.24   80.48   19.12    94.94     5.06   
% Solved                 100.00  100.00  100.00  100.00    97.44    85.68   
Avg. % gap                  NaN     NaN     NaN     NaN     0.43     0.86   
Avg. time                  0.28    0.26    2.09    7.06   309.79  1302.17   
Std. dev. of time          0.80    0.58    6.56   53.70  1303.10  2555.60   
Avg. iterations            4.04    3.73    7.37    6.98    17.22    19.59   
Std. dev. of iterations    2.47    2.21    7.77    8.26    24.29    22.93   

                                                                               
                         100x2             150x2             200x2             
                         Wagner   Wilson   Wagner   Wilson   Wagner   Wilson   
% Best Performance         92.41     7.23    93.36     6.73    95.44     4.14  
% Solved                   80.96    50.56    67.52    45.50    63.46    35.25  
Avg. % gap                  1.06     1.69     1.23     1.64     1.07     0.63  
Avg. time                1571.97  3844.30  2625.08  4368.39  3033.07  5584.95  
Std. dev. of time        2804.57  3440.96  3315.93  3251.30  3340.53  2529.35  
Avg. iterations            28.56    21.47    27.70    13.82    29.13    10.80  
Std. dev. of iterations    30.02    15.47    28.50     7.92    45.18     3.80

In [34]:
# Save output table as HTML
pd.set_option('colheader_justify', 'center')   # FOR TABLE <th>

html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open(os.path.join(outputfolder_table, '2RPFS_instance_stats.html'), 'w') as f:
    f.write(html_string.format(table=df_table3.to_html(classes='mystyle')))

## Table 4. Performance given all instances (SP=MIP)

In [58]:
model_stats_sp_mip = dict()
for model in ['Wagner']:
    model_stats_sp_mip[model] = dict()
    model_stats_sp_mip[model]['% Best Performance'] = calculate_perc_best_performance(df_rpfs_sp_mip, model)
    model_stats_sp_mip[model]['% Solved'] = calculate_perc_solved(df_rpfs_sp_mip, model)  # given all instances
    df_itype = df_rpfs_sp_mip.reset_index()
    instance_size_list = df_itype['instance_size'].unique().tolist()
    for instance_size in instance_size_list:
        model_stats_sp_mip[model]['% Solved '+ instance_size] = calculate_perc_solved(df_rpfs_sp_mip, model, None, instance_size)
    model_stats_sp_mip[model]['Avg. % gap'] = calculate_avg_perc_gap(df_rpfs_sp_mip, model)
    model_stats_sp_mip[model]['Median time'] = calculate_median_time(df_rpfs_sp_mip, model)
    model_stats_sp_mip[model]['Median iterations'] = calculate_median_iterations(df_rpfs_sp_mip, model)

In [59]:
model_stats_sp_mip

{'Wagner': {'% Best Performance': nan,
  '% Solved': 79.01,
  '% Solved 150x2': 86.45,
  '% Solved 200x2': 74.71,
  'Avg. % gap': 56.66,
  'Median time': 893.21,
  'Median iterations': 12.0}}

In [60]:
model_stats_sp_mip_df = pd.DataFrame.from_dict(model_stats_sp_mip)
model_stats_sp_mip_df.T

,% Best Performance,% Solved,% Solved 150x2,% Solved 200x2,Avg. % gap,Median iterations,Median time
Wagner,NaN,79.01,86.45,74.71,56.66,12.0,893.21


## Table 5. Performance per instance group and model (SP=MIP)

In [76]:
per_instance_stats_sp_mip = dict()
df_itype = df_rpfs_sp_mip.reset_index()
instance_size_list = df_itype['instance_size'].unique().tolist()
sp_type = 'MIP'
for instance_size in instance_size_list:
    print('Process instance_size = ', instance_size)
    df_instance = df_itype[df_itype['instance_size'] == instance_size]
    for model in ['Wagner']:
        per_instance_stats_sp_mip[(sp_type,instance_size,model)] = dict()
        per_instance_stats_sp_mip[(sp_type,instance_size,model)]['% Best Performance'] = calculate_perc_best_performance(df_instance, model)
        per_instance_stats_sp_mip[(sp_type,instance_size,model)]['% Solved'] = calculate_perc_solved(df_rpfs_sp_mip, model, None, instance_size)
        per_instance_stats_sp_mip[(sp_type,instance_size,model)]['Avg. % gap'] = calculate_avg_perc_gap(df_instance, model)
        per_instance_stats_sp_mip[(sp_type,instance_size,model)]['Avg. time'] = calculate_avg_time(df_instance, model)
        per_instance_stats_sp_mip[(sp_type,instance_size,model)]['Std. dev. of time'] = calculate_std_time(df_instance, model)
        per_instance_stats_sp_mip[(sp_type,instance_size,model)]['Avg. iterations'] = calculate_avg_iterations(df_instance, model)
        per_instance_stats_sp_mip[(sp_type,instance_size,model)]['Std. dev. of iterations'] = calculate_std_iterations(df_instance, model)

Process instance_size =  150x2
Process instance_size =  200x2


In [77]:
pd.set_option('display.max_columns', None)
df_table5 = pd.DataFrame.from_dict(per_instance_stats_sp_mip)
df_table5

MIP            
                         150x2    200x2  
                         Wagner   Wagner 
% Best Performance           NaN      NaN
% Solved                   86.45    74.71
Avg. % gap                 11.62    70.58
Avg. time                1119.35  3133.70
Std. dev. of time        2425.14  2792.97
Avg. iterations            21.57    23.35
Std. dev. of iterations    35.78    28.93

In [78]:
per_instance_stats2 = { your_key: per_instance_stats[your_key] for your_key in [('DP', '150x2', 'Wagner'), ('DP', '200x2', 'Wagner')] }
df_table_dp = pd.DataFrame.from_dict(per_instance_stats2)
df_table_dp.join(df_table5)

DP               MIP            
                         150x2    200x2    150x2    200x2  
                         Wagner   Wagner   Wagner   Wagner 
% Best Performance         93.36    95.44      NaN      NaN
% Solved                   67.52    63.46    86.45    74.71
Avg. % gap                  1.23     1.07    11.62    70.58
Avg. time                2625.08  3033.07  1119.35  3133.70
Std. dev. of time        3315.93  3340.53  2425.14  2792.97
Avg. iterations            27.70    29.13    21.57    23.35
Std. dev. of iterations    28.50    45.18    35.78    28.93